# 지도학습
- 분류문제 (classification): 범주형 종속변수
- 회귀문제 (regression): 연속형 종속변수

## k-최근접 이웃 (k-Nearest Neighbor) 알고리즘
- 비교 대상이 되는 데이터 포인트 주변에 가장 가까이 존재하는 k개의 데이터와 비교해 가장 가까운 데이터 종류로 판별
- 판별 결과는 k값에 따라 달라짐
- 타깃이 연속형 숫자인 경우에는 주변 k개의 데이터의 평균값으로 예측하는 방법 사용
- 학습 과정에서 게으른 학습(lazy learning) 방법 사용

In [1]:
# 데이터 불러오기
from sklearn import datasets
raw_iris = datasets.load_iris()

In [2]:
# 피처/타깃 데이터 지정
X = raw_iris.data
y = raw_iris.target

In [3]:
# 트레이닝/ 테스트 데이터 분할
from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)

In [5]:
# 데이터 표준화
from sklearn.preprocessing import StandardScaler
std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

In [6]:
# 데이터 학습
from sklearn.neighbors import KNeighborsClassifier
clf_knn = KNeighborsClassifier()
clf_knn.fit(X_tn_std, y_tn)

KNeighborsClassifier()

In [7]:
# 데이터 예측
knn_pred = clf_knn.predict(X_te_std)
print(knn_pred)

[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 2]


In [8]:
# 정확도 평가
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, knn_pred)
print(accuracy)

0.9736842105263158


In [9]:
# confusion matrix 확인
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, knn_pred) # 실제 타깃값과 예측값 넣어줌
print(conf_matrix)

[[13  0  0]
 [ 0 15  1]
 [ 0  0  9]]


In [10]:
# 분류 리포트 확인
from sklearn.metrics import classification_report
class_report = classification_report(y_te, knn_pred)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38



## 선형 회귀 분석 (linear regression)
- feature 데이터와 target 데이터 간의 선형 관게 파악하는 알고리즘
- 가중치 w 구해야 (y=wx+b)
- 각 가중치 요소 하나하나 = 파라미터, 파라미터 값에 따라 예측값 달라짐
- 최소제곱법 통해 구함

### 릿지 회귀 분석 (L2 제약식) - ridge regression
- 제약식: 제곱의 형태
- 람다: penalty strength
> - 계수(coefficient)의 사이즈 조절
> - 정규식의 크기(amount) 조절
> - 0에 가까워질수록 최소 제곱 추정량(least squared estimator)에 가까워짐
> - 무한대에 가까워질수록 릿지 해는 0에 가까워짐. 즉, 상수항(intercept)만 남은 모형에 가까워짐

##### 릿치 추정량의 편향성
- 릿지 회귀 분석의 성질: 편향 존재 (bias)
- 즉, 릿지 추정량의 기댓값은 실제 파라미터(parameter)와 동일하지 않음

### 라쏘 회귀 분석 (L1 제약식) - lasso regression
- 제약식: 절댓값의 형태
- 라쏘 추정량은 w1=0인 부분 -> 변수 선택(variable selection)에 사용될 수 있음
- 릿지 회귀 분석과 달리 라쏘 추정량은 확실한 형태가 존재하지 않음 (closed form solution)

##### 제약식의 일반화 표현

### 엘라스틱 넷 (Elastic Net)
- ridge + lasso

In [11]:
# 데이터 불러오기 (보스턴 집값 데이터 예측)
from sklearn import datasets
raw_boston = datasets.load_boston()

# 피처, 타깃 데이터 지정
X = raw_boston.data
y = raw_boston.target

# 트레이닝/테스트 데이터 분할
from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=1)

# 데이터 표준화
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
std_scaler.fit(X_tn)
X_tn_std = std_scaler.transform(X_tn)
X_te_std = std_scaler.transform(X_te)

In [13]:
# 데이터 학습
from sklearn.linear_model import LinearRegression
clf_lr = LinearRegression()
clf_lr.fit(X_tn_std, y_tn)

LinearRegression()

In [14]:
# 선형 회귀 분석 계수, 상수항 확인
print(clf_lr.coef_)
print(clf_lr.intercept_)

[-1.07145146  1.34036243  0.26298069  0.66554537 -2.49842551  1.97524314
  0.19516605 -3.14274974  2.66736136 -1.80685572 -2.13034748  0.56172933
 -4.03223518]
22.344591029023768


In [15]:
# 데이터 학습 (L2 제약식 적용, 릿지 회귀 분석)
from sklearn.linear_model import Ridge
clf_ridge = Ridge(alpha=1) # alpha 값은 반드시 양수(positive)여야 하며, 값이 클수록 강한 제약식을 의미 (기본값 1로 설정되어 있음)
clf_ridge.fit(X_tn_std, y_tn)

# 릿지 회귀 분석 계수, 상수항 확인
print(clf_ridge.coef_)
print(clf_ridge.intercept_)

[-1.05933451  1.31050717  0.23022789  0.66955241 -2.45607567  1.99086611
  0.18119169 -3.09919804  2.56480813 -1.71116799 -2.12002592  0.56264409
 -4.00942448]
22.344591029023768


In [17]:
# 데이터 학습 (L1 제약식 적용, 라쏘 회귀 분석)
from sklearn.linear_model import Lasso
clf_lasso = Lasso(alpha=0.01) # alpha 값은 제약의 정도를 나타내며 1을 디폴트 값으로 가짐
clf_lasso.fit(X_tn_std, y_tn)

# 라쏘 회귀 분석 계수, 상수항 확인
print(clf_lasso.coef_)
print(clf_lasso.intercept_)

[-1.04326518  1.27752711  0.1674367   0.66758228 -2.41559964  1.99244179
  0.14733958 -3.09473711  2.46431135 -1.60552274 -2.11046422  0.55200229
 -4.00809905]
22.344591029023768


In [19]:
# 데이터 학습 (엘라스틱 넷)
from sklearn.linear_model import ElasticNet
clf_elastic = ElasticNet(alpha=0.01, l1_ratio=0.01) 
# alpha 값은 L1 제약식과 L2 제약식의 크기의 합, l1_ratio는 전체 제약의 크기인 alpha 값에서 L1 제약이 차지하는 비율 의미
clf_elastic.fit(X_tn_std, y_tn)

# 엘라스틱 넷 계수, 상수항 확인
print(clf_elastic.coef_)
print(clf_elastic.intercept_)

[-1.02916603  1.23681955  0.15236504  0.67859622 -2.34646781  2.02965524
  0.14575132 -2.98592423  2.32013379 -1.48829485 -2.09271972  0.56506801
 -3.9495281 ]
22.344591029023768


In [20]:
# 데이터 예측
pred_lr = clf_lr.predict(X_te_std)
pred_ridge = clf_ridge.predict(X_te_std)
pred_lasso = clf_lasso.predict(X_te_std)
pred_elastic = clf_elastic.predict(X_te_std)

In [21]:
# 모형 평가 - R 제곱값 (0과 1 사잇값 가지며 높을수록 좋은 성능 의미)
from sklearn.metrics import r2_score
print(r2_score(y_te, pred_lr))
print(r2_score(y_te, pred_ridge))
print(r2_score(y_te, pred_lasso))
print(r2_score(y_te, pred_elastic))

0.7789410172622858
0.7789704562726605
0.7787621490259894
0.7787876079239252


In [22]:
# 모형 평가 - MSE (작을수록 좋은 성능 의미)
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_te, pred_lr))
print(mean_squared_error(y_te, pred_ridge))
print(mean_squared_error(y_te, pred_lasso))
print(mean_squared_error(y_te, pred_elastic))

21.89776539604949
21.894849212618745
21.91548381050483
21.912961890936874


## 로지스틱 회귀 분석
- 분류 문제
- 시그모이드 함수 형태
- y축 값이 0과 1로 제한되어 있음
- 가중치의 부호에 따라 로지스틱 회귀 모형의 형태가 달라짐

In [23]:
# 데이터 불러오기 (유방암 예측하기)
from sklearn import datasets
raw_cancer = datasets.load_breast_cancer()

# 피처, 타깃 데이터 지정
X = raw_cancer.data
y = raw_cancer.target

# 트레이닝/테스트 데이터 분할
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)

# 데이터 표준화
from sklearn.preprocessing import StandardScaler
std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

In [24]:
# 데이터 학습
from sklearn.linear_model import LogisticRegression
clf_logi_l2 = LogisticRegression(penalty='l2') 
# 선형 회귀 분석과 마찬가지로 제약식 적용할 수 있음. l1, l2, elasticnet, none 중 하나 입력해서 사용
clf_logi_l2.fit(X_tn_std, y_tn)

LogisticRegression()

In [25]:
# 로지스틱 회귀 분석 추정 계수
print(clf_logi_l2.coef_)
print(clf_logi_l2.intercept_)

[[-0.29792942 -0.58056355 -0.3109406  -0.377129   -0.11984232  0.42855478
  -0.71131106 -0.85371164 -0.46688191  0.11762548 -1.38262136  0.0899184
  -0.94778563 -0.94686238  0.18575731  0.99305313  0.11090349 -0.3458275
   0.20290919  0.80470317 -0.91626377 -0.91726667 -0.8159834  -0.86539197
  -0.45539191  0.10347391 -0.83009341 -0.98445173 -0.5920036  -0.61086989]]
[0.02713751]


In [26]:
# 데이터 예측
pred_logistic = clf_logi_l2.predict(X_te_std)
print(pred_logistic)

[0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 0 1 0 1 1 0 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0
 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 1 0 0 0 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1
 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 0 0 1 1 1 0]


In [27]:
# 클래스 확률로 예측
pred_proba = clf_logi_l2.predict_proba(X_te_std) # 해당 클래스에 속할 확률 출력
print(pred_proba)

[[9.98638613e-01 1.36138656e-03]
 [3.95544804e-02 9.60445520e-01]
 [1.30896362e-03 9.98691036e-01]
 [1.24473354e-02 9.87552665e-01]
 [2.44132101e-04 9.99755868e-01]
 [4.50491513e-03 9.95495085e-01]
 [1.13985968e-04 9.99886014e-01]
 [1.82475894e-03 9.98175241e-01]
 [9.67965506e-05 9.99903203e-01]
 [1.75222878e-06 9.99998248e-01]
 [1.76572612e-01 8.23427388e-01]
 [8.24119135e-02 9.17588087e-01]
 [9.66067493e-06 9.99990339e-01]
 [5.39343196e-01 4.60656804e-01]
 [3.98187854e-01 6.01812146e-01]
 [9.95762760e-01 4.23724017e-03]
 [2.75612083e-03 9.97243879e-01]
 [9.99997097e-01 2.90271401e-06]
 [9.99926506e-01 7.34935682e-05]
 [9.99999997e-01 2.78313939e-09]
 [9.98738365e-01 1.26163489e-03]
 [9.81405399e-01 1.85946008e-02]
 [1.77902039e-02 9.82209796e-01]
 [9.65876713e-04 9.99034123e-01]
 [9.99464578e-01 5.35421808e-04]
 [6.73385015e-04 9.99326615e-01]
 [5.50833875e-05 9.99944917e-01]
 [9.69828919e-01 3.01710813e-02]
 [1.62119075e-03 9.98378809e-01]
 [9.99997821e-01 2.17867101e-06]
 [6.005712

In [28]:
# 정밀도 평가
from sklearn.metrics import precision_score
print(precision_score(y_te, pred_logistic))

0.9666666666666667


In [29]:
# confusion matrix 확인
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_te, pred_logistic))

[[50  3]
 [ 3 87]]


In [30]:
# 분류 리포트 확인
from sklearn.metrics import classification_report
print(classification_report(y_te, pred_logistic))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94        53
           1       0.97      0.97      0.97        90

    accuracy                           0.96       143
   macro avg       0.96      0.96      0.96       143
weighted avg       0.96      0.96      0.96       143



## 나이브 베이즈 (Naive Bayes)
- 조건부 독립(conditional independence)인 피처 가정
- 베이즈 이론 기반

In [1]:
# 데이터 불러오기 (와인 종류 구분)
from sklearn import datasets
raw_wine = datasets.load_wine()

# 피처, 타깃 데이터 지정
X = raw_wine.data
y = raw_wine.target

# 트레이닝/테스트 데이터 분할
from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)

# 데이터 표준화
from sklearn.preprocessing import StandardScaler
std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

In [2]:
# 데이터 학습
from sklearn.naive_bayes import GaussianNB
clf_gnb = GaussianNB()
clf_gnb.fit(X_tn_std, y_tn)

GaussianNB()

In [3]:
# 데이터 예측
pred_gnb = clf_gnb.predict(X_te_std)
pred_gnb

array([0, 2, 1, 0, 1, 1, 0, 2, 1, 1, 2, 2, 0, 0, 2, 1, 0, 0, 2, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 2, 0, 0, 1, 0, 0, 0, 2, 1, 1, 2, 0, 0, 1, 1,
       1])

In [4]:
# 리콜 평가
from sklearn.metrics import recall_score
recall = recall_score(y_te, pred_gnb, average='macro')
recall

0.9523809523809524

In [10]:
# confusion matrix 확인
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_gnb)
print(conf_matrix)

[[16  0  0]
 [ 2 18  1]
 [ 0  0  8]]


In [9]:
# 분류 리포트 확인
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_gnb)
print(class_report)

              precision    recall  f1-score   support

           0       0.89      1.00      0.94        16
           1       1.00      0.86      0.92        21
           2       0.89      1.00      0.94         8

    accuracy                           0.93        45
   macro avg       0.93      0.95      0.94        45
weighted avg       0.94      0.93      0.93        45



## 의사 결정 나무 (decision tree)
- root node, leaf node, decision node, ancestor node, parent note, child node, sibling node, level, depth, degree
- 이해하기 쉽다는 장점, 다만 오버피팅 된다는 단점 => 보완하기 위해 random forest 사용

### 엔트로피
- 불순도(impurity) 정도 측정, 낮을수록 좋음 (불순도=노드에 서로 다른 데이터가 얼마나 섞여 있는가를 의미)
- 각 노드별 엔트로피 이용해 테스트 전체의 성능 평가 (테스트 성능 점수도 낮을수록 좋음)

### 지니계수 (gini index)
- 불순도 측정하는 또 다른 방법
- 데이터 셋에서 랜덤으로 선택한 데이터에 임의로 라벨링을 했을 때 틀릴 확률

### 회귀 나무
- 의사결정나무의 피처가 연속형 변수인 경우
>  각 피처 값을 따로 테스트 기준으로 정하고 테스트
- 타깃 데이터가 연속형 변수인 경우
> 회귀 나무 모형 (regression tree) 사용,
> 예측값을 각 항목 평균값으로

In [11]:
# 데이터 불러오기 (와인 종류 분류)
from sklearn import datasets
raw_wine = datasets.load_wine()

# 피처, 타깃 데이터 지정
X = raw_wine.data
y = raw_wine.target

# 트레이닝/테스트 데이터 분할
from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)

# 데이터 표준화
from sklearn.preprocessing import StandardScaler
std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

In [13]:
# 데이터 학습
from sklearn import tree
clf_tree = tree.DecisionTreeClassifier(random_state=0)
clf_tree.fit(X_tn_std, y_tn)

DecisionTreeClassifier(random_state=0)

In [14]:
# 데이터 예측
pred_tree = clf_tree.predict(X_te_std)
print(pred_tree)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 2 0 1 0 1 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 1 0 1 1 1]


In [16]:
# f1 score 평가
from sklearn.metrics import f1_score
f1 = f1_score(y_te, pred_tree, average='macro')
print(f1)

0.9349141206870346


In [17]:
# confusion matrix 확인
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_tree)
print(conf_matrix)

# 분류 리포트 확인
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_tree)
print(class_report)

[[14  2  0]
 [ 0 20  1]
 [ 0  0  8]]
              precision    recall  f1-score   support

           0       1.00      0.88      0.93        16
           1       0.91      0.95      0.93        21
           2       0.89      1.00      0.94         8

    accuracy                           0.93        45
   macro avg       0.93      0.94      0.93        45
weighted avg       0.94      0.93      0.93        45



# 서포트 벡터 머신 (SVM: Support Vector Machine)
- 서포트 벡터를 기준으로 클래스 판별
- 중심선과 경계선을 이용해 데이터 구분 (경계선 = 서포트 벡터)
- 마진(margin): 서포트 벡터 간 너비(width) - 최대화하는 것이 목표
- 데이터 분포의 특성(i.e., 공분산) 이용하지 않는 것이 특징

#### 소프트 마진(soft margin)
- 잘못 분류된 데이터를 '어느 정도' 허용
- 슬랙 변수(slack variable) 사용: 해당 데이터가 잘 분류되었는지를 나타내는 변수 --> 데이터 포인트가 잘못된 영역에 속하고 서포트 벡터로부터 멀리 떨어져 있을 수록 값 커짐

## 커널 서포트 벡터 머신(Kernel Support Vector Machine)
- 피처 공간을 변경한 후 SVM 적용
- 커널 함수(kernel function) 사용: dth Degree polynomial, Radial basis, Neural network 등 (1) 적분값이 1이며 (2) 원점을 중심으로 대칭이고 (3) non-negative인 함수
- 커널 함수 바꿈으로써 여러 가지 학습 모형 생성 가능

## 서포트 벡터 회귀 (Support Vector Regression)
- 앞선 svm들은 분류 모형. SVR은 회귀 모형
- 서포트 벡터 기준으로 회귀
- 중심선은 회귀 직선이 됨

In [9]:
# 데이터 불러오기
from sklearn import datasets
raw_wine = datasets.load_wine()

# 피처, 타깃 데이터 지정
X = raw_wine.data
y = raw_wine.target

# 트레이닝/테스트 데이터 분할
from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)

# 데이터 표준화
from sklearn.preprocessing import StandardScaler
std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

In [10]:
# 데이터 학습
from sklearn import svm
# kernel: 서포트 벡터 머신의 커널 정하는 옵션 (linear, poly, rbf, sigmoid, precomputed 등)
clf_svm_lr = svm.SVC(kernel='linear', random_state=0) #SVC 메소드 사용해서 모형 설정 (분류 문제). 회귀 문제라면 SVR 메소드 사용
clf_svm_lr.fit(X_tn_std, y_tn)

SVC(kernel='linear', random_state=0)

In [13]:
# 데이터 예측
pred_svm = clf_svm_lr.predict(X_te_std)
print(pred_svm)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


In [14]:
# 정확도 평가
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_svm)
print(accuracy)

1.0


# Cross Validation 실습

hyperparameter tuning을 위해 크로스 밸리데이션 기법 활용

In [18]:
# 데이터 불러오기
from sklearn import datasets
raw_wine = datasets.load_wine()

# 피처, 타깃 데이터 지정
X = raw_wine.data
y = raw_wine.target

# 트레이닝/테스트 데이터 분할
from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)

# 데이터 표준화
from sklearn.preprocessing import StandardScaler
std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

In [19]:
# 그리드 서치

from sklearn import svm
# stratifid k fold = 일반적인 k-fold와 달리 라벨링의 비율을 유지하면서 데이터 추출하는 방법
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

param_grid = {'kernel':('linear', 'rbf'),
             'C':[0.5, 1, 10, 100]}

# 크로스 밸리데이션 방법 설정
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
# 학습 시킬 기본 모형 설정
svc = svm.SVC(random_state=0)
# 그리드서치 수행 (학습시킬모형, 파라미터, 크로스밸리데이션 방법, 모형평가방법)
grid_cv = GridSearchCV(svc, param_grid, cv=kfold, scoring='accuracy')
grid_cv.fit(X_tn_std, y_tn)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=SVC(random_state=0),
             param_grid={'C': [0.5, 1, 10, 100], 'kernel': ('linear', 'rbf')},
             scoring='accuracy')

In [20]:
# 그리드 서치 결과 확인
grid_cv.cv_results_

{'mean_fit_time': array([0.00443034, 0.00417032, 0.00434422, 0.00357814, 0.00357924,
        0.00379252, 0.00414205, 0.00664115]),
 'std_fit_time': array([0.00240352, 0.00075232, 0.00061365, 0.00048402, 0.00050013,
        0.00146622, 0.00043628, 0.00132584]),
 'mean_score_time': array([0.00056343, 0.00139608, 0.00171628, 0.00157576, 0.00100584,
        0.0013876 , 0.00130763, 0.00329261]),
 'std_score_time': array([4.42708786e-04, 4.87264552e-04, 7.41993072e-04, 4.86283140e-04,
        1.49223368e-05, 4.79921025e-04, 4.02898974e-04, 9.68751921e-04]),
 'param_C': masked_array(data=[0.5, 0.5, 1, 1, 10, 10, 100, 100],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['linear', 'rbf', 'linear', 'rbf', 'linear', 'rbf',
                    'linear', 'rbf'],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=ob

In [21]:
# 그리드 서치 결과 확인(DF)
import numpy as np
import pandas as pd
np.transpose(pd.DataFrame(grid_cv.cv_results_))

,0,1,2,3,4,5,6,7
mean_fit_time,0.00443,0.00417,0.004344,0.003578,0.003579,0.003793,0.004142,0.006641
std_fit_time,0.002404,0.000752,0.000614,0.000484,0.0005,0.001466,0.000436,0.001326
mean_score_time,0.000563,0.001396,0.001716,0.001576,0.001006,0.001388,0.001308,0.003293
std_score_time,0.000443,0.000487,0.000742,0.000486,0.000015,0.00048,0.000403,0.000969
param_C,0.5,0.5,1,1,10,10,100,100
param_kernel,linear,rbf,linear,rbf,linear,rbf,linear,rbf
params,"{'C': 0.5, 'kernel': 'linear'}","{'C': 0.5, 'kernel': 'rbf'}","{'C': 1, 'kernel': 'linear'}","{'C': 1, 'kernel': 'rbf'}","{'C': 10, 'kernel': 'linear'}","{'C': 10, 'kernel': 'rbf'}","{'C': 100, 'kernel': 'linear'}","{'C': 100, 'kernel': 'rbf'}"
split0_test_score,0.888889,0.962963,0.888889,0.925926,0.888889,0.925926,0.888889,0.925926
split1_test_score,0.962963,1.0,0.962963,0.962963,0.962963,0.962963,0.962963,0.962963
split2_test_score,0.925926,0.962963,0.925926,0.962963,0.925926,0.962963,0.925926,0.962963


In [22]:
# best score & hyperparameter
grid_cv.best_score_

0.9774928774928775

In [23]:
grid_cv.best_params_

{'C': 0.5, 'kernel': 'rbf'}

In [24]:
# 최종모형
clf = grid_cv.best_estimator_
print(clf)

SVC(C=0.5, random_state=0)


In [25]:
# 크로스 밸리데이션 스코어 확인(1)
from sklearn.model_selection import cross_validate # 여러가지 지표 확인
metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
cv_scores = cross_validate(clf, X_tn_std, y_tn, cv=kfold, scoring=metrics)

In [26]:
cv_scores

{'fit_time': array([0.00598407, 0.00499105, 0.00302649, 0.00294971, 0.00498676]),
 'score_time': array([0.01141024, 0.01013231, 0.004987  , 0.00793982, 0.00997376]),
 'test_accuracy': array([0.96296296, 1.        , 0.96296296, 0.96153846, 1.        ]),
 'test_precision_macro': array([0.96296296, 1.        , 0.96969697, 0.96969697, 1.        ]),
 'test_recall_macro': array([0.96666667, 1.        , 0.96296296, 0.95833333, 1.        ]),
 'test_f1_macro': array([0.9628483 , 1.        , 0.96451914, 0.96190476, 1.        ])}

In [27]:
# 크로스 밸리데이션 스코어 확인(2)
from sklearn.model_selection import cross_val_score # 한가지 지표 확인
cv_score = cross_val_score(clf, X_tn_std, y_tn, cv=kfold, scoring='accuracy')
print(cv_score)

[0.96296296 1.         0.96296296 0.96153846 1.        ]


In [28]:
print(cv_score.mean()) # 정확도의 평균
print(cv_score.std()) # 정확도의 표준편차

0.9774928774928775
0.01838434849561446


In [29]:
# 예측
pred_svm = clf.predict(X_te_std)
print(pred_svm)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


In [30]:
# 정확도
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_svm)
print(accuracy)

1.0


In [31]:
# confusion matrix 확인
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_svm)
print(conf_matrix)

[[16  0  0]
 [ 0 21  0]
 [ 0  0  8]]


In [32]:
# 분류 리포트 확인
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_svm)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

